# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы. Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — она сделана ранее.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.


__Описание данных:__

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:

    сalls — количество звонков,
    
    minutes — суммарная длительность звонков в минутах,
    
    messages — количество sms-сообщений,
    
    mb_used — израсходованный интернет-трафик в Мб,
    
    is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).
    
    
__Краткий план действий:__ открыть и изучить данные, выделить признаки и тагрет, разбить данные на выборки(учебную, проверочную, тестовую), инициализировать и обучить модели, проверить их на тестовой выборке.     

## Откроем и изучим файл

In [1]:
# импортируем библиотеки

import pandas as pd 
import numpy as np

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
try:
    data = pd.read_csv('/Users/alex/Downloads/users_behavior.csv')
except:
    data = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [4]:
# взглянем поближе
data.head(10)   

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


In [22]:
data.head(30)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


In [6]:
# проверим на наличие явных дубликатов
data.duplicated().sum()

0

In [7]:
# полностью пустых/незаполненных строк - нет
data.isna().sum()

calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64

In [8]:
data['is_ultra'].value_counts()

0    2229
1     985
Name: is_ultra, dtype: int64

__Промежуточный вывод__

У нас имеется датафрейм с 3214 стоками, имеется 2 различных типа данных float и int(что логично т.к мы знаем из описания там хранятся значения либо 1, либо 0 - обозначение тарифного плана), пользователей с тарифом "Смарт" почти в 2 раза больше чем пользователей с тарифом "Ультра"

## Разбейте данные на выборки

Сначала выделим наши признаки и таргет, затем разодьем наш дф на 3 части - учебную/проверочную/тестовую.

In [9]:
# Выделим признаки - все столбцы за исключением нашего таргета is_ultra(его мы удалим)

features = data.drop(['is_ultra'], axis = 1)

# Выделим целевой признак(наш таргет) is_ultra.

target = data['is_ultra']

# Создадим переменную и присвоим ей значение random_state для дальнейшего использования

rdst = 123

In [10]:
# Отделим 20% тестовых данных от самого дф

features_data, features_test, target_data, target_test = train_test_split(
    features, target, test_size = 0.2, random_state = rdst)

# Оставшиеся данные поделим на соответсвующие обучающие и проверочные(валидационные) данные(пропорции будут - 75/25)

features_train, features_valid, target_train, target_valid = train_test_split(
    features_data, target_data, test_size = 0.25, random_state = rdst)


In [11]:
# проверим соотношение тренировочной/валидной/тестовой частей к нашему первоначальному дф'у

print(f'Соотношение тренировочной части к дф: {round(features_train.shape[0] / data.shape[0], 2)}')
print(f'Соотношение валидной части к дф: {round(features_valid.shape[0] / data.shape[0], 2)}')
print(f'Соотношение тестовой части к дф: {round(features_test.shape[0] / data.shape[0], 2)}')

Соотношение тренировочной части к дф: 0.6
Соотношение валидной части к дф: 0.2
Соотношение тестовой части к дф: 0.2


__Промежуточный вывод__

На данном этапе мы проделели следующие:
- Выделили признаки и записали их переменные features и target;
- Сначала отделили тестовую часть, затем из оставшиеся данные разделили на обучающую и проверочную части;
- Вывели их соотношение.

## Исследуйте модели

Исследуйте качество разных моделей, меняя гиперпараметры. Кратко напишите выводы исследования.

Модель - __Логистическая регрессия__

In [13]:
#инициализируем модель LogisticRegression
model_logr = LogisticRegression(random_state = rdst, solver = 'lbfgs', max_iter = 1000) 
model_logr.fit(features_train, target_train)

# посчитаем качество модели
result_logr = model_logr.score(features_valid, target_valid)
print(f'Accuracy модели логистической регрессии на валидационной выборке: {result_logr.round(3)}')

Accuracy модели логистической регрессии на валидационной выборке: 0.734


Модель - __Дерево решений__

In [14]:
# напишем цикл для глубины дерева

best_depth_dtc = 0
best_result_dtc = 0
best_model_dtc = None

for depth in range(1, 15):
    #инициализируем модель DecisionTreeClassifier
    model_dtc = DecisionTreeClassifier(random_state = rdst, max_depth = depth, criterion = 'gini')
    model_dtc.fit(features_train,target_train)
    predictions = model_dtc.predict(features_valid)
    # посчитаем качество модели
    result_dtc = accuracy_score(target_valid, predictions)
    if result_dtc > best_result_dtc:
        best_result_dtc = result_dtc
        best_depth_dtc = depth
        best_model_dtc = model_dtc
        
print(f'Accuracy лучшей модели равна: {best_result_dtc.round(3)}, глубина дерева: {best_depth_dtc}')

Accuracy лучшей модели равна: 0.795, глубина дерева: 9


Модель - __Случайный лес__

In [15]:
# Напишем цикл для модели Случайный лес, выведем лучшую модель.

best_depth_rfc = 0
best_result_rfc = 0
best_est_rfc = 0
best_model_rfc = None

for est in range(1, 16):
    for depth in range(1, 16):
        #инициализируем модель RandomForestClassifier
        model_rfc = RandomForestClassifier(random_state = rdst,
                                           max_depth = depth,
                                           n_estimators = est,
                                           criterion = 'gini')
        # обучим модель
        model_rfc.fit(features_train, target_train)
        result_rfc = model_rfc.score(features_valid, target_valid)
        if result_rfc > best_result_rfc:
            best_result_rfc = result_rfc
            best_depth_rfc = depth
            best_est_rfc = est
            best_model_rfc = model_rfc

print(f'Accuracy лучшей модели равна: {best_result_rfc.round(4)}, количество деревьев: {best_est_rfc}, глубина дерева: {best_depth_rfc}')

Accuracy лучшей модели равна: 0.8103, количество деревьев: 8, глубина дерева: 8


__Краткий вывод по исследованию моделей__

Мы обучили 3 модели ("Логистическая регрессия", "Дерево решений", "Случайный лес") и проверили их на __валидационной(проверочной) выборке__.

Получили следующие показатели точности(accuracy):
- Логистическая регрессия, accuracy = 0.734;
- Дерево решений, accuracy = 0.795;
- Случайный лес, accuracy = 0.81

Как и ожидалось, лучший показатель точности у модели "Случайный лес".


## Проверьте модель на тестовой выборке

__Тест на модели "Логистическая регрессия"__

In [16]:
model_logr

LogisticRegression(max_iter=1000, random_state=123)

In [17]:
predictions_logr = model_logr.predict(features_test)
result_logr1 = accuracy_score(target_test, predictions_logr)

print(f'Accuracy модели логистической регрессии: \nНа валидационной выборке: {result_logr.round(3)} \nНа тестовой выборке: {result_logr1.round(3)}')

Accuracy модели логистической регрессии: 
На валидационной выборке: 0.734 
На тестовой выборке: 0.751


__Тест на модели "Дерево решений"__

In [18]:
best_model_dtc

DecisionTreeClassifier(max_depth=9, random_state=123)

In [19]:
predictions_test_dtc = best_model_dtc.predict(features_test)
result_dtc = accuracy_score(target_test, predictions_test_dtc)

print(f'Accuracy модели случайный лес: \nНа валидационной выборке: {best_result_dtc.round(3)} \nНа тестовой выборке: {result_dtc.round(3)}')

Accuracy модели случайный лес: 
На валидационной выборке: 0.795 
На тестовой выборке: 0.787


__Тест на модели "Случайный лес"__

In [20]:
best_model_rfc

RandomForestClassifier(max_depth=8, n_estimators=8, random_state=123)

In [21]:
predictions_test_rfc = best_model_rfc.predict(features_test)
result_rfc = accuracy_score(target_test, predictions_test_rfc)

print(f'Accuracy модели случайный лес: \nНа валидационной выборке: {best_result_rfc.round(4)} \nНа тестовой выборке: {result_rfc.round(3)}')

Accuracy модели случайный лес: 
На валидационной выборке: 0.8103 
На тестовой выборке: 0.795


__Вывод__

При проверке наших моделей на тестовой выборке, доля правильных ответов у моделей __дерево решений__ и __случайный лес__ немного уменьшилась, а у __логистической регрессии__ чуть-чуть увеличилась. Самый высокая доля правильных ответов у модели __Случайный лес__ - __accuracy__ составляет 0.8103 и 0.795.

Можно сказать, что наша модель может с 79%(81%) вероятностью подобрать подходящий тарифный план.

Вероятно имея более конкретизирующих данные (признаки), например, сколько пользователь тратит денег на связь, какие дополнительные опции он получит перейдя на тариф Ультра, безлемитные ли звонки, интернет или нет и тд....скорее всего, модель получилось бы улучшить.
